In [ ]:
from odata_client import ODataClient
from mcp_server import MCPServer
import asyncio
from mcp_server import (
    mcp_tool_entity_sets,
    get_schema,
    mcp_tool_metadata,
    resolve_entity_name,
    resolve_field_name,
    list_objects,
    find_object,
    create_object,
    update_object,
    delete_object,
    post_document,
    unpost_document,
    search_object,
    ensure_entity,
    create_document,
    get_fields,
    get_first_records,
    find_record,
    create_record,
    post_record,
    add_product_service,
    get_records_by_date_range,
    get_records_with_expand,
    get_aggregated_data
)


In [8]:
BASE_URL = 'http://192.168.18.113/TEST19/odata/standard.odata'
USERNAME = 'nikita'
PASSWORD = 'nikita2000'
VERIFY_SSL = 'false'

print('Creds:')
print(BASE_URL)
print(USERNAME)
print(PASSWORD)
print(VERIFY_SSL)

_server = MCPServer(base_url=BASE_URL, username=USERNAME, password=PASSWORD, verify_ssl=VERIFY_SSL)

Creds:
http://192.168.18.113/TEST19/odata/standard.odata
nikita
nikita2000
false


2025-08-19 20:44:50,102 [INFO] mcp_server: 1C OData metadata cache warmed up successfully


In [4]:
await mcp_tool_entity_sets()

2025-08-19 19:48:21,322 [INFO] mcp_server: MCP tool mcp_tool_entity_sets called with {"args": [], "kwargs": {}}
2025-08-19 19:48:21,421 [INFO] mcp_server: MCP tool mcp_tool_entity_sets result {"http_code": 200, "http_message": "OK", "entity_sets": ["Catalog_Контрагенты", "Catalog_Контрагенты_КонтактнаяИнформация", "Catalog_Контрагенты_ДополнительныеРеквизиты", "Catalog_Контрагенты_ИсторияКПП", "Catalog_Контрагенты_ИсторияНаименований", "Catalog_Контрагенты_ИсторияКонтактнойИнформации", "Catalog_Номенклатура", "Catalog_Номенклатура_ДополнительныеРеквизиты", "Catalog_Номенклатура_ИсторияВидаСтавкиНДС", "Catalog_Номенклатура_ИсторияПрослеживаемогоТовара", "Document_ПлатежноеПоручение", ...


{'http_code': 200,
 'http_message': 'OK',
 'entity_sets': ['Catalog_Контрагенты',
  'Catalog_Контрагенты_КонтактнаяИнформация',
  'Catalog_Контрагенты_ДополнительныеРеквизиты',
  'Catalog_Контрагенты_ИсторияКПП',
  'Catalog_Контрагенты_ИсторияНаименований',
  'Catalog_Контрагенты_ИсторияКонтактнойИнформации',
  'Catalog_Номенклатура',
  'Catalog_Номенклатура_ДополнительныеРеквизиты',
  'Catalog_Номенклатура_ИсторияВидаСтавкиНДС',
  'Catalog_Номенклатура_ИсторияПрослеживаемогоТовара',
  'Document_ПлатежноеПоручение',
  'Document_ПоступлениеТоваровУслуг',
  'Document_ПоступлениеТоваровУслуг_Оборудование',
  'Document_ПоступлениеТоваровУслуг_ОбъектыСтроительства',
  'Document_ПоступлениеТоваровУслуг_Товары',
  'Document_ПоступлениеТоваровУслуг_Услуги',
  'Document_ПоступлениеТоваровУслуг_ВозвратнаяТара',
  'Document_ПоступлениеТоваровУслуг_ЗачетАвансов',
  'Document_ПоступлениеТоваровУслуг_АгентскиеУслуги',
  'Document_ПоступлениеТоваровУслуг_ОсновныеСредства',
  'Document_ПоступлениеТова

In [23]:
await get_entities()

2025-08-19 20:53:33,148 [INFO] mcp_server: MCP tool get_entities called with {"args": [], "kwargs": {}}
2025-08-19 20:53:33,463 [INFO] mcp_server: MCP tool get_entities result {"http_code": 200, "http_message": "OK", "odata_error_code": null, "odata_error_message": null, "entities": ["Catalog_Контрагенты", "Catalog_Контрагенты_КонтактнаяИнформация", "Catalog_Контрагенты_ДополнительныеРеквизиты", "Catalog_Контрагенты_ИсторияКПП", "Catalog_Контрагенты_ИсторияНаименований", "Catalog_Контрагенты_ИсторияКонтактнойИнформации", "Catalog_Номенклатура", "Catalog_Номенклатура_ДополнительныеРеквизиты", "Catalog_Номенклатура_ИсторияВидаСтавкиНДС", "Catalog_Номенклатура_ИсторияПро...


{'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None,
 'entities': ['Catalog_Контрагенты',
  'Catalog_Контрагенты_КонтактнаяИнформация',
  'Catalog_Контрагенты_ДополнительныеРеквизиты',
  'Catalog_Контрагенты_ИсторияКПП',
  'Catalog_Контрагенты_ИсторияНаименований',
  'Catalog_Контрагенты_ИсторияКонтактнойИнформации',
  'Catalog_Номенклатура',
  'Catalog_Номенклатура_ДополнительныеРеквизиты',
  'Catalog_Номенклатура_ИсторияВидаСтавкиНДС',
  'Catalog_Номенклатура_ИсторияПрослеживаемогоТовара',
  'Document_ПлатежноеПоручение',
  'Document_ПоступлениеТоваровУслуг',
  'Document_ПоступлениеТоваровУслуг_Оборудование',
  'Document_ПоступлениеТоваровУслуг_ОбъектыСтроительства',
  'Document_ПоступлениеТоваровУслуг_Товары',
  'Document_ПоступлениеТоваровУслуг_Услуги',
  'Document_ПоступлениеТоваровУслуг_ВозвратнаяТара',
  'Document_ПоступлениеТоваровУслуг_ЗачетАвансов',
  'Document_ПоступлениеТоваровУслуг_АгентскиеУслуги',
  'Document_ПоступлениеТовар

In [9]:
await get_schema("Catalog_Контрагенты")

2025-08-19 20:44:52,522 [INFO] mcp_server: MCP tool get_schema called with {"args": ["Catalog_Контрагенты"], "kwargs": {}}
2025-08-19 20:44:52,617 [INFO] mcp_server: MCP tool get_schema result {"http_code": 200, "http_message": "OK", "odata_error_code": null, "odata_error_message": null, "schema": {"entity_type": "StandardODATA.Catalog_Контрагенты", "properties": {"Ref_Key": {"type": "Edm.Guid", "nullable": false}, "Predefined": {"type": "Edm.Boolean", "nullable": true}, "PredefinedDataName": {"type": "Edm.String", "nullable": true}, "DataVersion": {"type": "Edm.String", "nullable": true}, "Description": {"type": "Edm.String", "nullable": true}, "Code": {"type": "Edm.String", "nullabl...


{'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None,
 'schema': {'entity_type': 'StandardODATA.Catalog_Контрагенты',
  'properties': {'Ref_Key': {'type': 'Edm.Guid', 'nullable': False},
   'Predefined': {'type': 'Edm.Boolean', 'nullable': True},
   'PredefinedDataName': {'type': 'Edm.String', 'nullable': True},
   'DataVersion': {'type': 'Edm.String', 'nullable': True},
   'Description': {'type': 'Edm.String', 'nullable': True},
   'Code': {'type': 'Edm.String', 'nullable': True},
   'Parent_Key': {'type': 'Edm.Guid', 'nullable': True},
   'IsFolder': {'type': 'Edm.Boolean', 'nullable': True},
   'DeletionMark': {'type': 'Edm.Boolean', 'nullable': True},
   'НаименованиеПолное': {'type': 'Edm.String', 'nullable': True},
   'ОбособленноеПодразделение': {'type': 'Edm.Boolean', 'nullable': True},
   'ЮридическоеФизическоеЛицо': {'type': 'Edm.String', 'nullable': True},
   'СтранаРегистрации_Key': {'type': 'Edm.Guid', 'nullable': True},
   'Гол

In [10]:
await mcp_tool_metadata()

2025-08-19 20:45:19,601 [INFO] mcp_server: MCP tool mcp_tool_metadata called with {"args": [], "kwargs": {}}
2025-08-19 20:45:19,891 [INFO] mcp_server: MCP tool mcp_tool_metadata result {"http_code": 200, "http_message": "OK", "entity_sets": {"Catalog_Контрагенты": {"entity_type": "StandardODATA.Catalog_Контрагенты", "properties": {"Ref_Key": {"type": "Edm.Guid", "nullable": false}, "Predefined": {"type": "Edm.Boolean", "nullable": true}, "PredefinedDataName": {"type": "Edm.String", "nullable": true}, "DataVersion": {"type": "Edm.String", "nullable": true}, "Description": {"type": "Edm.String", "nullable": true}, "Code": {"type": "Edm.String", "nullable": true}, "Parent_Key": {...


{'http_code': 200,
 'http_message': 'OK',
 'entity_sets': {'Catalog_Контрагенты': {'entity_type': 'StandardODATA.Catalog_Контрагенты',
   'properties': {'Ref_Key': {'type': 'Edm.Guid', 'nullable': False},
    'Predefined': {'type': 'Edm.Boolean', 'nullable': True},
    'PredefinedDataName': {'type': 'Edm.String', 'nullable': True},
    'DataVersion': {'type': 'Edm.String', 'nullable': True},
    'Description': {'type': 'Edm.String', 'nullable': True},
    'Code': {'type': 'Edm.String', 'nullable': True},
    'Parent_Key': {'type': 'Edm.Guid', 'nullable': True},
    'IsFolder': {'type': 'Edm.Boolean', 'nullable': True},
    'DeletionMark': {'type': 'Edm.Boolean', 'nullable': True},
    'НаименованиеПолное': {'type': 'Edm.String', 'nullable': True},
    'ОбособленноеПодразделение': {'type': 'Edm.Boolean', 'nullable': True},
    'ЮридическоеФизическоеЛицо': {'type': 'Edm.String', 'nullable': True},
    'СтранаРегистрации_Key': {'type': 'Edm.Guid', 'nullable': True},
    'ГоловнойКонтраген

In [12]:
await resolve_entity_name("Контрагенты")

2025-08-19 20:45:58,280 [INFO] mcp_server: MCP tool resolve_entity_name called with {"args": ["Контрагенты"], "kwargs": {}}
2025-08-19 20:45:58,578 [INFO] mcp_server: MCP tool resolve_entity_name result {"resolved": "Catalog_Контрагенты", "http_code": 200, "http_message": "OK", "odata_error_code": null, "odata_error_message": null}


{'resolved': 'Catalog_Контрагенты',
 'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None}

In [19]:
await resolve_field_name("Catalog_Контрагенты", "наименование")

2025-08-19 20:48:05,609 [INFO] mcp_server: MCP tool resolve_field_name called with {"args": ["Catalog_Контрагенты", "наименование"], "kwargs": {}}
2025-08-19 20:48:05,695 [INFO] mcp_server: MCP tool resolve_field_name result {"resolved": "Description", "http_code": 200, "http_message": "OK", "odata_error_code": null, "odata_error_message": null}


{'resolved': 'Description',
 'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None}

In [22]:
await list_objects("Catalog_Контрагенты", filters={"КПП":'732901001'})

2025-08-19 20:51:01,552 [INFO] mcp_server: MCP tool list_objects called with {"args": ["Catalog_Контрагенты"], "kwargs": {"filters": {"КПП": "732901001"}}}
2025-08-19 20:51:01,828 [INFO] mcp_server: MCP tool list_objects result {"http_code": 200, "http_message": "OK", "odata_error_code": null, "odata_error_message": null, "last_id": null, "data": [{"Ref_Key": "a728dfad-c3d6-11ed-abc5-00155d58e400", "DataVersion": "AAAAAAAXnUc=", "DeletionMark": false, "Parent_Key": "30f1a174-9af8-11ed-abc4-00155d58e400", "IsFolder": false, "Code": "БП-000224", "Description": "СИТИДОРС ООО", "НаименованиеПолное": "ООО \"СИТИДОРС\"", "ОбособленноеПодразделение": false, "ЮридическоеФизическоеЛицо": "ЮридическоеЛицо", "СтранаРегистрации_Ke...


{'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None,
 'last_id': None,
 'data': [{'Ref_Key': 'a728dfad-c3d6-11ed-abc5-00155d58e400',
   'DataVersion': 'AAAAAAAXnUc=',
   'DeletionMark': False,
   'Parent_Key': '30f1a174-9af8-11ed-abc4-00155d58e400',
   'IsFolder': False,
   'Code': 'БП-000224',
   'Description': 'СИТИДОРС ООО',
   'НаименованиеПолное': 'ООО "СИТИДОРС"',
   'ОбособленноеПодразделение': False,
   'ЮридическоеФизическоеЛицо': 'ЮридическоеЛицо',
   'СтранаРегистрации_Key': '45ff0068-36df-11e9-80ed-0050569f2e9f',
   'ГоловнойКонтрагент_Key': 'a728dfad-c3d6-11ed-abc5-00155d58e400',
   'ИНН': '7328065121',
   'КПП': '732901001',
   'КодПоОКПО': '',
   'ДокументУдостоверяющийЛичность': '',
   'ОсновнойБанковскийСчет_Key': 'a728dfae-c3d6-11ed-abc5-00155d58e400',
   'УдалитьОсновнойДоговорКонтрагента_Key': '00000000-0000-0000-0000-000000000000',
   'ОсновноеКонтактноеЛицо_Key': 'a728dfbe-c3d6-11ed-abc5-00155d58e400',
   'Комментарий'

In [ ]:
await get_fields()